# Carbon Use Efficiency of Marine Heterotrophs

In [1]:
import cobra
import os
import pandas as pd

In [10]:
import gem2cue.utils
import gem2cue.calculate_cue

## Test

In [17]:
# Test with Ecoli
ecoli_strain = gem2cue.utils.Strain('e_coli',
	cobra.io.read_sbml_model('test/test_files/EC_core_flux1.xml'),
	None,
	None,
)

In [18]:
ecoli_strain

In [19]:
ecoli_exp = gem2cue.utils.Experiment(ecoli_strain)

In [20]:
ecoli_exp.CUE()

In [21]:
ecoli_exp.cue

0.6198361114965819

In [22]:
# Test with other test model
test_strain = gem2cue.utils.Strain('iIT341',
	cobra.io.read_sbml_model('test/test_files/iIT341.xml'),
	None,
	None,
)
test_exp = gem2cue.utils.Experiment(test_strain)
test_exp.CUE()
test_exp.cue

0.8396726347751939

## Set-up

In [2]:
# Read in the meta-data file as a data frame
meta_df = pd.read_csv('IAMM_models/strain_metadata.tsv', sep='\t')

In [3]:
meta_df

,strain,RefSeq_assembly_accession,model_file,gc_content,genome_length
0,Bork,GCF_000009365.1,Bork-LB.xml,0.547278,3120143
1,Dies,GCF_000300005.1,Dies-LB.xml,0.616316,4928223
2,Hot1A3,GCF_001578515.1,Hot1A3-LB.xml,0.446714,4801807
3,BS11,GCF_000299995.1,BS11-LB.xml,0.446078,4480507
4,DE,GCF_000310085.1,DE-LB.xml,0.448687,4480937
5,Cstat,GCF_001561975.1,Cstat-LB.xml,0.548907,2862233
6,DFL12,GCF_000018145,DFL12-LB.xml,0.655246,4417868
7,Kuro,GCF_000425705.1,Kuro-LB.xml,0.470156,3850372
8,HP15,GCF_000166295.1,HP15-LB.xml,0.568599,4651725
9,MMB1,GCF_000192865.1,MMB1-LB.xml,0.441275,4684316


In [8]:
# List the file name of all of the models I made from NCBI faas
# The file name is the assembly accession number
avail_models = [os.path.splitext(f)[0] for f in os.listdir(os.path.join('IAMM_models', 'faa_from_ncbi'))]

In [12]:
# For each row, make a Strain object with the model and metadata from the file
strain_list = []
for index, row in meta_df.iterrows():
    # Only make the strain if we have the model for it
    if row['RefSeq_assembly_accession'] in avail_models:
        strain_list.append(gem2cue.utils.Strain(row['strain'],
                                                cobra.io.read_sbml_model(os.path.join('IAMM_models', 'faa_from_ncbi', row['RefSeq_assembly_accession']) + '.xml'),
                                                row['gc_content'],
                                                row['genome_length']))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Analysis

### Experiment 0: Unconstrained

In [13]:
# Make an experiment for each Strain
unconstrained_experiments = [gem2cue.utils.Experiment(strain) for strain in strain_list]

In [16]:
# Calculate CUE for all of the Experiments
for exp in unconstrained_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

KeyboardInterrupt: 

In [15]:
# Pull out all of the CUE results
[exp.cue for exp in unconstrained_experiments]

[0.953807672517817,
 0.9381487345462173,
 0.9341523289656387,
 0.8966798579175352,
 0.8961445703250691,
 0.9154150701003737,
 0.9319850981515869,
 0.8985223400827563,
 0.9259476445012083]

### Experiment 1: Glucose
The media on the COBRApy model already has 1000 for the glucose, so just run it
unconstrained.

In [ ]:
print(strain_list[0].model.medium)

In [ ]:
# Make the medium
gluc_media = None

In [ ]:
# Make an experiment for each Strain
glucose_experiments = [gem2cue.utils.Experiment(strain) for strain in strain_list]

In [ ]:
# Calculate CUE for all of the Experiments
for exp in glucose_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

In [ ]:
[exp.cue for exp in glucose_experiments]

### Experiment 2: Acetate

In [ ]:
# Copy the medium from one of the models
ace_media = strain_list[0].model.medium
# It already has acetate, just need to remove glucose
ace_media['EX_glc__D_e'] = 0

In [ ]:
# Make an experiment for each Strain
ace_experiments = [gem2cue.utils.Experiment(strain, ace_media) for strain in strain_list]

In [ ]:
# Calculate CUE for all of the Experiments
for exp in ace_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

In [ ]:
[exp.cue for exp in ace_experiments]

## Visualization